# Sonification of Bleeding with Bank of Filters

First Meeting (2019-06-11, Sasan and Thomas) @CITEC, TH proposed Filter-bank for feature generation
* The idea is to use a bank of different low-pass filters to create increasingly smooth signals
* these filtered signals serve as source for identifying key moments to anchor sound events
* which then create a multiscale data-driven complex grain structure of the raw instantaneous bleeding data.
* note that the limit of filtering with a cutoff-frequency towards 0 yields the integrated signal.

## Imports

In [ ]:
from scipy import signal
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt 
import copy

In [ ]:
%matplotlib notebook

## Load Data and create filter bank signals

In [ ]:
df = pd.read_csv('log_refactored_correction_factor.csv', na_values=['no info', '.'], delimiter=',')
df_indexed = df.reset_index(drop=False)
index = df_indexed['index']
delta = df_indexed['Delta']
volume = df_indexed['Blood Accumulated']

In [ ]:
# help(signal.butter)

In [ ]:
import scipy.interpolate
dfn = scipy.interpolate.interp1d(index, delta) #'previous')
delta_resampled = dfn(np.linspace(0, len(index)-1, len(index)*20))
# plt.plot(delta_resampled, "r.-")

In [ ]:
srcsig = delta_resampled
N = srcsig.shape[0]
NF = 15  # number of filters to use
sr = 20
dsf = np.zeros((N, NF))
cfs = np.array(1+np.arange(NF))/(3*NF)  # To do: experiment with exponentially tuned filter frequencies
order = 1 # 2... create oscillations into negative...

for i, cf in enumerate(cfs):
    b, a = signal.butter(order, cf,fs=sr)
    zi = signal.lfilter_zi(b, a)
    z, _ = signal.lfilter(b, a, srcsig, zi = zi*delta[0])
    dsf[:, i] = copy.copy(z)

# plot the data
#fig = plt.figure(figsize=(15,4))
plt.plot(srcsig, 'k.-', lw=0.2)
plt.plot(dsf, color='r', lw=0.3);
plt.ylim(-0.2,5); 
#plt.xlim(0,60*sr); 
plt.grid()

In [ ]:
plt.hist(srcsig, 80)
plt.semilogy()

## Event-based Sonification of filtered data (min/max/threshold cut-throughs...)

In [ ]:
import sc3nb as scn
import time
sc = scn.startup()

In [ ]:
%sc FreqScope(400, 300)

In [ ]:
# filterbank argeggio
for i, r in enumerate(dsf):
    if i<3:
        continue
    for j, v in enumerate(r):
        if np.argmax(dsf[i-2:i+1,j]) == 1: 
            freq = scn.midicps(scn.linlin(j, 0, NF-1, 50, 90)) # warum j und nicht v?
            sc.msg("/s_new", ["s1", -1, 1, 1, "freq", freq, "dur", 0.05*j, "num", 1]) # was sind die arguments von s_new? z.b. -1,1,1
    time.sleep(0.05)

In [ ]:
# 1-filter melody
idx_prev = [-1, -1, -1]
scale = [0,2,4,7,9,12,14,16,19,21,24]
for i, r in enumerate(dsf):
    c = 1
    v = r[c]
    idx = int(scn.linlin(v, 0, 5, 0, 11))
    if idx != idx_prev[0]:
        freq = scn.midicps(36 + scale[idx])
        sc.msg("/s_new", ["s1", -1, 1, 1, "freq", freq, "dur", 2.5, "num", 5])
        idx_prev[0] = idx

    c = 6
    v = r[c]
    idx = scn.clip(int(scn.linlin(v, 0, 5, 0, 11)), 0, 11)
    if idx != idx_prev[1]:
        freq = scn.midicps(60 + scale[idx])
        sc.msg("/s_new", ["s1", -1, 1, 1, "freq", freq, "dur", 0.4, "num", 1])
        idx_prev[1] = idx
        
    c = NF-1
    v = r[c]
    idx = int(scn.linlin(v, 0, 5, 0, 11))
    if idx != idx_prev[2]:
        freq = scn.midicps(84 + scale[idx])
        sc.msg("/s_new", ["s1", -1, 1, 1, "freq", freq, "dur", 0.1, "num", 1])
        idx_prev[2] = idx
    time.sleep(0.05)